# Imports permettant de générer les CSV

In [1]:
import json
import time
import numpy as np
import pandas as pd
from itertools import zip_longest

# Import Sélénium et drivers de chrome

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
import selenium

# Import Beatiful Soup 4

In [3]:
import requests
from bs4 import BeautifulSoup
import lxml

# Paramétrage de Selenium

In [4]:
# Setup the desired capabilities of the browser
options = webdriver.ChromeOptions()

# Remove the notifications
options.add_experimental_option("excludeSwitches", ["enable-logging"])

# Make the browser headless(without GUI)
options.add_argument('--headless')
# Give the browser a size
options.add_argument("--window-size=1920,1080")
# Disable the browser from using the GPU
options.add_argument("--disable-gpu")

# UserAgent for the browser
useAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
options.add_argument(f'user-agent={useAgent}')

# Giving the driver the capabilities to use the chrome driver and to use the chrome browser and to use the options

url = "https://www.fifa.com/fifa-world-ranking/men?dateId="

# Fonction pour scraper les tables de classement
### But :
Permet de charger la page et de le transformer en dataframe

- input : url
- output : dataframe contenant les données RK(classement), Country, Total Points, Previous Points, +/-

ouvre une session chrome, charge la page, ferme la session chrome, transforme le tableau HTML en data frame

# Phase 1 et 2

In [5]:
def scrapTable(url,options=options):
    pd.set_option('display.max_rows', 500)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.width', 1000)

    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get(url)
    time.sleep(3)

    # récupération de la page souorce
    html = driver.page_source
    driver.close() #fermeture de session chrome

    # récupération du tableau HTML
    soup = BeautifulSoup(html, 'lxml')
    htmlTable = soup.find_all('table')

    # transformation en dataframe
    table = pd.read_html(str(htmlTable))
    frames = [table[0]]
    table = pd.concat(frames, ignore_index=True)
    table = table.drop(table.columns[[1, 6, 7]], axis=1)
    table.insert(1, "Country", "")
    table['Country'] = table['Team'].str[-3:]
    table['Team'] = table['Team'].str[:-3]

    #retour du dataframe
    return  table

# Fonction pour récuperer le JSON intégré entre les balises \<script>
### But :
Permet de charger la page et de recuperer les données contenu entre les balises \<script>\</script> 

- input : url
- output : JSON object avec les données contenu dans date (id et text qui n'est autre que la date)

ouvre une session chrome, charge la page, ferme la session chrome, parcours le "JSON" pour récuperer l'information utile

In [6]:
def scrapJsonNextData(url,options=options):
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()
    driver.get(url)

    # récupération de la page souorce
    html = driver.page_source
    driver.close() #fermeture de session chrome

    # récupération de l'bjet JSON caché dans la page
    soup = BeautifulSoup(html, 'html.parser')
    res = soup.find_all('script',{"id":"__NEXT_DATA__"})
    script = res[0].text
    
    #dransformation d'une partie du JSON en dictionnaire
    json_object = json.loads(script)
    part = json_object['props']['pageProps']['pageData']['ranking']['dates']

    # retour du dictionnaire
    return part

# routine principale
### But :
Permet de charger les pages et de recuperer les différents classement et les dauvegarder en CSV

- récupere les données du "JSON"
- boucle sur le chargement des pages à scraper et transforme les tableau HTML en CSV




# Phase 3

In [7]:
# récupération du dictionnaire
id_dates = scrapJsonNextData(url)

#boucle itératif basé sur le dictionnaire
for id_date in id_dates:
    print(id_date['id'],id_date['text'])

    #récupération en dataframe du tableau des pages à scraper
    df = scrapTable(url+str(id_date['id']))
    text=id_date['text']

    #sauvegarde du dataframe en CSV
    path='Men s Ranking '+text+'.csv'.replace(' ','_')
    df.to_csv(path,sep=';',encoding="UTF-8", index=False)

id13792 6 Oct 2022
id13750 25 Aug 2022
id13687 23 Jun 2022
id13603 31 Mar 2022
id13554 10 Feb 2022
id13505 23 Dec 2021
id13471 19 Nov 2021
id13442 21 Oct 2021
id13407 16 Sept 2021
id13372 12 Aug 2021
id13295 27 May 2021
id13245 7 Apr 2021
id13197 18 Feb 2021
id13127 10 Dec 2020
id13113 26 Nov 2020
id13078 22 Oct 2020
id13043 17 Sept 2020
id12884 16 Jul 2020
id12883 11 Jun 2020
id12882 9 Apr 2020
id12833 20 Feb 2020
id12770 19 Dec 2019
id12749 28 Nov 2019
id12714 24 Oct 2019
id12679 19 Sept 2019
id12623 25 Jul 2019
id12582 14 Jun 2019
id12511 4 Apr 2019
id12455 7 Feb 2019
id12406 20 Dec 2018
id12385 29 Nov 2018
id12350 25 Oct 2018
id12315 20 Sept 2018
id12280 16 Aug 2018
id12252 1 Jul 2018
id12210 7 Jun 2018
id12189 17 May 2018
id12154 12 Apr 2018
id12126 15 Mar 2018
id12098 15 Feb 2018
id12070 18 Jan 2018
id12042 21 Dec 2017
id12014 23 Nov 2017
id11976 16 Oct 2017
id11944 14 Sept 2017
id11909 10 Aug 2017
id11874 6 Jul 2017
id11839 1 Jun 2017
id11811 4 May 2017
id11783 6 Apr 2017
id1175

In [8]:
print (selenium.__version__)

4.6.0
